In [54]:
import pandas as pd 
import re
import os
import sys
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,  ReduceLROnPlateau
from tensorflow.keras.regularizers import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import *
from datetime import datetime
from sklearn.model_selection import train_test_split
from pandas import DataFrame as df
from openpyxl import load_workbook
from tqdm.notebook import tqdm
from keras.preprocessing.text import Tokenizer
from konlpy.tag import Mecab
from keras.datasets import reuters
from keras.layers import Embedding, Dense, LSTM, Bidirectional 
from keras.models import Sequential 
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
mecab = Mecab()

In [55]:
corpos_df = pd.read_csv("/home/snsd/projects/system/vodabi/vodabi_pattern_tester/corpos_df.csv")

In [56]:
corpos_df.tail()

,Unnamed: 0,Filename,PATTERN,KEYWORD,BANT
392227,392227,0605_01032135259,네 차장님께서 간단히 검토하신 다음에 문의사항 있으시면 그쪽으로 연락을 좀 주시면 ...,빈 값,4
392228,392228,0605_01032135259,네 알겠습니다,빈 값,4
392229,392229,0605_01032135259,네 차장님 통화 감사 드립니다,빈 값,4
392230,392230,0605_01032135259,네 감사합니다,빈 값,4
392231,392231,0605_01032135259,네,빈 값,4


In [57]:
len(corpos_df)

392232

In [58]:
corpos_df_train = corpos_df[:353008]

In [59]:
corpos_df_test = corpos_df[353008:]

In [60]:
sample_set = corpos_df_train

In [61]:
#okt, mecab 불용어 처리 
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다', "예","예예","예예예","네","네네","네 네","네네네", "감사합니다","알겠습니다","예 알겠습니다", "네 알겠습니다", "예 감사합니다", "네 감사합니다"]
x_sample_data = [] 
for sentence in tqdm(sample_set['PATTERN']): 
    temp_X = [] 
    #temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = mecab.morphs(sentence) # 토큰화 
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    x_sample_data.append(temp_X) 

In [62]:
from keras.preprocessing.text import Tokenizer

In [63]:
vocab_size = 35000
tokenizer = Tokenizer(num_words = vocab_size) 
tokenizer.fit_on_texts(x_sample_data) 
sequences_sample = tokenizer.texts_to_sequences(x_sample_data)


In [64]:
test_set = corpos_df_test

In [65]:
#okt, #mecab 불용어 처리
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다', "예","예예","예예예","네","네네","네 네","네네네", "감사합니다","알겠습니다","예 알겠습니다", "네 알겠습니다", "예 감사합니다", "네 감사합니다"]
x_test_data = [] 
for sentence in tqdm(test_set['PATTERN']): 
    temp_X = []
    #temp_X = okt.morphs(sentence,stem=True) # 토큰화 
    temp_X = mecab.morphs(sentence) # 토큰화 
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    x_test_data.append(temp_X) 

In [66]:
vocab_size = 35000 # 데이터셋에서 가장 빈도 높은 몇개의 단어만 사용할지
tokenizer = Tokenizer(num_words = vocab_size) 
tokenizer.fit_on_texts(x_sample_data) 
sequences_test = tokenizer.texts_to_sequences(x_test_data)

In [67]:
y_sample = sample_set['BANT']

In [68]:
#none 포함일때
y_sample_1 = []
for i in range(len(y_sample)):
    if y_sample.iloc[i] == 0: 
        y_sample_1.append([1, 0, 0, 0, 0])
    elif y_sample.iloc[i] == 1: 
        y_sample_1.append([0, 1, 0, 0, 0])
    elif y_sample.iloc[i] == 2: 
        y_sample_1.append([0, 0, 1, 0, 0])
    elif y_sample.iloc[i] == 3: 
        y_sample_1.append([0, 0, 0, 1, 0])
    elif y_sample.iloc[i] == 4: 
        y_sample_1.append([0, 0, 0, 0, 1])

In [69]:
y_sample_1 = np.array(y_sample_1)

In [70]:
y_sample_1

array([[0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       ...,
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1]])

In [71]:
y_test = test_set['BANT']

In [72]:
# none 포함 일 때
y_test_1 = []
for i in range(len(y_test)):
    if y_test.iloc[i] == 0: 
        y_test_1.append([1, 0, 0, 0, 0])
    elif y_test.iloc[i] == 1: 
        y_test_1.append([0, 1, 0, 0, 0])
    elif y_test.iloc[i] == 2: 
        y_test_1.append([0, 0, 1, 0, 0])
    elif y_test.iloc[i] == 3: 
        y_test_1.append([0, 0, 0, 1, 0])
    else: #y_test.iloc[i] == 4: 
        y_test_1.append([0, 0, 0, 0, 1])

In [73]:
y_test_1 = np.array(y_test_1)
y_test_1

array([[0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       ...,
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1]])

In [74]:
max_len = 230 #패턴 최대 길이
pad_X_train = pad_sequences(sequences_sample, maxlen=max_len)
pad_X_test = pad_sequences(sequences_test, maxlen=max_len)

In [75]:
#modeling

In [76]:
class MultiHeadAttention(Layer):
    def __init__(self, embedding_dim, num_heads=8): 
        super(MultiHeadAttention, self).__init__()
        self.embedding_dim = embedding_dim # d_model
        self.num_heads = num_heads

        assert embedding_dim % self.num_heads == 0

        self.projection_dim = embedding_dim // num_heads
        self.query_dense = Dense(embedding_dim)
        self.key_dense = Dense(embedding_dim)
        self.value_dense = Dense(embedding_dim)
        self.dense = Dense(embedding_dim)
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'embedding_dim' : self.embedding_dim,
            'num_heads' : self.num_heads,
            'projection_dim' : self.projection_dim,
            'query_dense' : self.query_dense,
            'key_dense' : self.key_dense,
            'value_dense' : self.value_dense,
            'dense' : self.dense
        })

    def scaled_dot_product_attention(self, query, key, value):
        matmul_qk = tf.matmul(query, key, transpose_b=True) #matmul은 행렬 곱해주는 삼수
        depth = tf.cast(tf.shape(key)[-1], tf.float32) #소수점을 버림
        logits = matmul_qk / tf.math.sqrt(depth)
        attention_weights = tf.nn.softmax(logits, axis=-1)
        output = tf.matmul(attention_weights, value)
        return output, attention_weights

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]

        # (batch_size, seq_len, embedding_dim)
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)

        # (batch_size, num_heads, seq_len, projection_dim)
        query = self.split_heads(query, batch_size)  
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        scaled_attention, _ = self.scaled_dot_product_attention(query, key, value)
        # (batch_size, seq_len, num_heads, projection_dim)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  

        # (batch_size, seq_len, embedding_dim)
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.embedding_dim))
        outputs = self.dense(concat_attention)
        return outputs

In [77]:
class TransformerBlock(Layer):
    def __init__(self, embedding_dim, num_heads, dff, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(embedding_dim, num_heads)
        self.ffn = Sequential(
            [Dense(dff, activation="relu"),
             Dense(embedding_dim),]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)
        
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'att' : self.att,
            'ffn' : self.ffn,
            'layernorm1' : self.layernorm1,
            'layernorm2' : self.layernorm2,
            'dropout1' : self.dropout1,
            'dropout2' : self.dropout2
        })

    def call(self, inputs, training):
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [78]:
class TokenAndPositionEmbedding(Layer):
    def __init__(self, max_len, vocab_size, embedding_dim):
        super().__init__()
        self.token_emb = Embedding(vocab_size, embedding_dim)
        self.pos_emb = Embedding(max_len, embedding_dim)
        
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'token_emb' : self.token_emb,
            'pos_emb' : self.pos_emb,
        })
        return config

    def call(self, x):
        max_len = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=max_len, delta=1) # - tf.range(<start>,<limit>,<delta>,<name>): start~limit 사이에서 delta 간격으로 뽑아낸 정수들의 리스트를 생성함
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [79]:
embedding_dim = 32  # Embedding size for each token
num_heads = 4  # Number of attention heads
dff = 32  # Hidden layer size in feed forward network inside transformer

inputs = Input(shape=(max_len,))
embedding_layer = TokenAndPositionEmbedding(max_len, vocab_size, embedding_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embedding_dim, num_heads, dff)
x = transformer_block(x)
x = GlobalAveragePooling1D()(x)
x = Dropout(0.1)(x)
x = Dense(8, activation="relu")(x)
x = Dropout(0.1)(x)
outputs = Dense(5, activation="softmax")(x)

model = Model(inputs=inputs, outputs=outputs)

In [80]:
path = '/home/snsd/projects/data/dacon/model/checkpoint'

In [81]:
y_sample_1

array([[0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       ...,
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1]])

In [82]:
pad_X_train

array([[  0,   0,   0, ..., 440,  91,  52],
       [  0,   0,   0, ...,  91,   1, 168],
       [  0,   0,   0, ...,   9,  51,  44],
       ...,
       [  0,   0,   0, ...,   0,   0,  89],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,  33]], dtype=int32)

In [83]:
y_train = y_sample_1
y_test = y_test_1

In [84]:
x_train = pad_X_train
x_test = pad_X_test

In [92]:
print(len(y_train))
print(len(y_test))
print(len(x_train))
print(len(x_test))

353008
39224
353008
39224


In [87]:
ckpt_1 = 'tf_chkpoint.ckpt'
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
mc = ModelCheckpoint(filepath =  os.path.join(path, ckpt_1), monitor = 'val_accuracy', save_best_only = True, mode = 'max',verbose = 1, save_weights_only=True)
history = model.fit(x_train, y_train, batch_size=32, epochs=4, validation_split=0.2, callbacks = [mc])

Epoch 1/4
8825/8826 [============================>.] - ETA: 0s - loss: 0.1835 - accuracy: 0.9325
Epoch 00001: val_accuracy improved from -inf to 0.93340, saving model to /home/snsd/projects/data/dacon/model/checkpoint/tf_chkpoint.ckpt
8826/8826 [==============================] - 716s 81ms/step - loss: 0.1835 - accuracy: 0.9325 - val_loss: 0.1645 - val_accuracy: 0.9334
Epoch 2/4
8825/8826 [============================>.] - ETA: 0s - loss: 0.1426 - accuracy: 0.9395
Epoch 00002: val_accuracy did not improve from 0.93340
8826/8826 [==============================] - 722s 82ms/step - loss: 0.1426 - accuracy: 0.9395 - val_loss: 0.1664 - val_accuracy: 0.9323
Epoch 3/4
8825/8826 [============================>.] - ETA: 0s - loss: 0.1332 - accuracy: 0.9413
Epoch 00003: val_accuracy did not improve from 0.93340
8826/8826 [==============================] - 718s 81ms/step - loss: 0.1332 - accuracy: 0.9413 - val_loss: 0.1592 - val_accuracy: 0.9282
Epoch 4/4
8825/8826 [============================>.] 

In [88]:
print("\n 테스트 정확도 : %.2f" % (model.evaluate(x_test,y_test)[1]*100)) #에포크 50 배치 64 -> 86.17 / 에포크 80 배치 64 -> 0.8632

1226/1226 [==============================] - 26s 21ms/step - loss: 0.1560 - accuracy: 0.9366

 테스트 정확도 : 93.66


In [89]:
predict = model.predict(x_test)

In [90]:
import numpy as np 
predict_labels = np.argmax(predict, axis=1) 
original_labels = np.argmax(y_test, axis=1)

In [91]:
pattern = []
original_label = []
predict_label = []
for i in range(1000): 
    print("패턴 : ", test_set['PATTERN'].iloc[i], "/\t 원래 라벨 : ", original_labels[i], "/\t예측한 라벨 : ", predict_labels[i])
    pattern.append(test_set['PATTERN'].iloc[i])
    original_label.append(original_labels[i])
    predict_label.append(predict_labels[i])
#corpos_df['PATTERN'][313785:].iloc[i]

패턴 :  에스 /	 원래 라벨 :  4 /	예측한 라벨 :  4
패턴 :  티 이 케이 /	 원래 라벨 :  4 /	예측한 라벨 :  4
패턴 :  티 이 케이 /	 원래 라벨 :  4 /	예측한 라벨 :  4
패턴 :  씨 오 쩜 케이 알 이요 /	 원래 라벨 :  4 /	예측한 라벨 :  4
패턴 :  씨 오 쩜 케이 알 이시고 /	 원래 라벨 :  4 /	예측한 라벨 :  4
패턴 :  예 /	 원래 라벨 :  4 /	예측한 라벨 :  4
패턴 :  위스텍이죠 그러시면 /	 원래 라벨 :  4 /	예측한 라벨 :  4
패턴 :  예 /	 원래 라벨 :  4 /	예측한 라벨 :  4
패턴 :  예 존함이 이자 /	 원래 라벨 :  4 /	예측한 라벨 :  4
패턴 :  남 /	 원래 라벨 :  4 /	예측한 라벨 :  4
패턴 :  남자  /	 원래 라벨 :  4 /	예측한 라벨 :  4
패턴 :  호요 /	 원래 라벨 :  4 /	예측한 라벨 :  4
패턴 :  호자  /	 원래 라벨 :  4 /	예측한 라벨 :  4
패턴 :  네 /	 원래 라벨 :  4 /	예측한 라벨 :  4
패턴 :  팀장님 되시나요 그러시면은 /	 원래 라벨 :  4 /	예측한 라벨 :  4
패턴 :  팀장님 되시나요 그러시면은 /	 원래 라벨 :  1 /	예측한 라벨 :  4
패턴 :  아니요 아니요 저는 아직 주임이요 /	 원래 라벨 :  4 /	예측한 라벨 :  4
패턴 :  아니요 아니요 저는 아직 주임이요 /	 원래 라벨 :  1 /	예측한 라벨 :  4
패턴 :  아 주임님이시고 /	 원래 라벨 :  4 /	예측한 라벨 :  4
패턴 :  네 /	 원래 라벨 :  4 /	예측한 라벨 :  4
패턴 :  예 주임님 뭐 지금 솔리드 웍스는 구매는 하셨는데 지금 사용 인원이 없다고 하셨는데  /	 원래 라벨 :  4 /	예측한 라벨 :  4
패턴 :  네  /	 원래 라벨 :  4 /	예측한 라벨 :  4
패턴 :  아 그러시면 주임님은 저 뭐 캐드만 좀 하시는건가요 아니

In [97]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [98]:
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])

In [99]:
# fit the model
mc = ModelCheckpoint(filepath =  os.path.join(path, ckpt_1), monitor = 'val_accuracy', save_best_only = True, mode = 'max',verbose = 1, save_weights_only=True)
history = model.fit(x_train, y_train, batch_size=32, epochs=4, validation_split=0.2, callbacks = [mc])

Epoch 1/4
8826/8826 [==============================] - 697s 79ms/step - loss: 0.1230 - acc: 0.9450 - f1_m: 0.9449 - precision_m: 0.9461 - recall_m: 0.9437 - val_loss: 0.1571 - val_acc: 0.9330 - val_f1_m: 0.9330 - val_precision_m: 0.9338 - val_recall_m: 0.9323
Epoch 2/4
8826/8826 [==============================] - 704s 80ms/step - loss: 0.1184 - acc: 0.9462 - f1_m: 0.9461 - precision_m: 0.9472 - recall_m: 0.9451 - val_loss: 0.1705 - val_acc: 0.9311 - val_f1_m: 0.9311 - val_precision_m: 0.9318 - val_recall_m: 0.9305
Epoch 3/4
8826/8826 [==============================] - 699s 79ms/step - loss: 0.1150 - acc: 0.9478 - f1_m: 0.9477 - precision_m: 0.9489 - recall_m: 0.9466 - val_loss: 0.1665 - val_acc: 0.9315 - val_f1_m: 0.9316 - val_precision_m: 0.9320 - val_recall_m: 0.9312
Epoch 4/4
8826/8826 [==============================] - 707s 80ms/step - loss: 0.1128 - acc: 0.9487 - f1_m: 0.9487 - precision_m: 0.9498 - recall_m: 0.9476 - val_loss: 0.1866 - val_acc: 0.9309 - val_f1_m: 0.9307 - val_pre

In [106]:
# evaluate the model
loss, accuracy, f1_score, precision, recall = model.evaluate(x_test, y_test, verbose=0)
print('loss: {:.3f}, accuracy: {:.3f}, precision: {:.3f}, recall: {:.3f}, f1_score: {:.3f}'.format(loss, accuracy, precision, recall, f1_score))

loss: 0.174, accuracy: 0.936, precision: 0.937, recall: 0.935, f1_score: 0.936
